In [1]:
import os

In [2]:
stations = os.listdir('Planets/New folder/Moon')

In [3]:
events_directions = []

In [4]:
for i in stations:
    events = os.listdir('Planets/New folder/Moon/' + i)
    for j in events:
        events_directions.append('Planets/New folder/Moon/' + i + '/' + j)

In [29]:
events_directions

['Planets/New folder/Moon/Mars/XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv',
 'Planets/New folder/Moon/Mars/XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv',
 'Planets/New folder/Moon/Mars/XB.ELYSE.02.BHV.2019-07-26HR12_evid0034.csv',
 'Planets/New folder/Moon/Mars/XB.ELYSE.02.BHV.2019-09-21HR03_evid0032.csv',
 'Planets/New folder/Moon/Mars/XB.ELYSE.02.BHV.2021-05-02HR01_evid0017.csv',
 'Planets/New folder/Moon/Mars/XB.ELYSE.02.BHV.2021-10-11HR23_evid0011.csv',
 'Planets/New folder/Moon/Mars/XB.ELYSE.02.BHV.2021-12-24HR22_evid0007.csv',
 'Planets/New folder/Moon/Mars/XB.ELYSE.02.BHV.2022-01-02HR04_evid0006.csv',
 'Planets/New folder/Moon/Mars/XB.ELYSE.02.BHV.2022-02-03HR08_evid0005.csv',
 'Planets/New folder/Moon/Mars/XB.ELYSE.02.BHV.2022-04-09HR22_evid0002.csv',
 'Planets/New folder/Moon/Mars/XB.ELYSE.02.BHV.2022-05-04HR23_evid0001.csv',
 'Planets/New folder/Moon/S12/xa.s12.00.mhz.1969-12-16HR00_evid00006.csv',
 'Planets/New folder/Moon/S12/xa.s12.00.mhz.1970-01-09HR00_evid00007.csv',
 'P

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from obspy.signal.trigger import classic_sta_lta, trigger_onset
from scipy.signal import butter, filtfilt, spectrogram
from matplotlib import cm

sampling_rate = 6.625  # Hz
sta_len = 120  # Short-term window in seconds
lta_len = 600  # Long-term window in seconds
threshold_on = 4  # Trigger on
threshold_off = 1.5  # Trigger off

def bandpass_filter(data, lowcut=0.01, highcut=0.5, fs=6.625, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

C:\Users\El-Wattaneya\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [6]:
event_times = []

In [7]:
def filterd_data(file):
    csv_file = file
    data_cat = pd.read_csv(csv_file,encoding='utf-8')
    try: 
        csv_times = np.array(data_cat['time_rel(sec)'])
        csv_data = np.array(data_cat['velocity(m/s)'])
    except:
        csv_times = np.array(data_cat['rel_time(sec)'])
        csv_data = np.array(data_cat['velocity(c/s)'])
        
    filtered_data = bandpass_filter(csv_data)
    cft = classic_sta_lta(filtered_data, int(sta_len * sampling_rate), int(lta_len * sampling_rate))
    triggers = trigger_onset(cft, threshold_on, threshold_off)
    f, t, Sxx = spectrogram(filtered_data, fs=sampling_rate, nperseg=1024)
    
    fig, axs = plt.subplots(3, 1, figsize=(12, 12))
    axs[0].plot(csv_times, filtered_data, label='Filtered Data')
    axs[0].set_title('Seismic Data (Filtered)')
    axs[0].set_xlabel('Time (s)')
    axs[0].set_ylabel('Velocity (m/s)')
    axs[0].legend()
    for i, (on, off) in enumerate(triggers):
        axs[0].axvline(x=csv_times[on], color='r', linestyle='--', label=f'Trigger {i+1} On' if i == 0 else "")
        axs[0].axvline(x=csv_times[off], color='purple', linestyle='--', label=f'Trigger {i+1} Off' if i == 0 else "")

    
    axs[1].plot(csv_times, cft, label='STA/LTA Characteristic Function')
    axs[1].set_title('STA/LTA Detection')
    axs[1].set_xlabel('Time (s)')
    axs[1].set_ylabel('Characteristic Function')
    axs[1].legend()
    for i, (on, off) in enumerate(triggers):
        axs[1].axvline(x=csv_times[on], color='r', linestyle='--', label=f'Trigger {i+1} On' if i == 0 else "")
        axs[1].axvline(x=csv_times[off], color='purple', linestyle='--', label=f'Trigger {i+1} Off' if i == 0 else "")

    
    pcm = axs[2].pcolormesh(t, f, Sxx, cmap=cm.jet, shading='gouraud')
    axs[2].set_title('Spectrogram of Seismic Data')
    axs[2].set_ylabel('Frequency [Hz]')
    axs[2].set_xlabel('Time [s]')
    axs[2].set_ylim([0, 1])  
    fig.colorbar(pcm, ax=axs[2],orientation='horizontal', label='Power ((m/s)^2/Hz)')
    
    plt.tight_layout()
    
    ff_name = file.split('/')[-1]
    output_image_file = 'images/' + f'{ff_name}.png'
    
    try: 
        for on, off in triggers:
            event_times.append((ff_name,
            data_cat[data_cat['time_rel(sec)'] == csv_times[on]][['time_abs(%Y-%m-%dT%H:%M:%S.%f)']].reset_index(drop = True)['time_abs(%Y-%m-%dT%H:%M:%S.%f)'][0], 
                                csv_times[on], csv_times[off],
                            output_image_file,))
    except:
        for on, off in triggers:
            event_times.append((ff_name,
            data_cat[data_cat['rel_time(sec)'] == csv_times[on]][['time(%Y-%m-%dT%H:%M:%S.%f)']].reset_index(drop = True)['time(%Y-%m-%dT%H:%M:%S.%f)'][0],  
                                csv_times[on], csv_times[off],
                            output_image_file,))
            
    plt.savefig(output_image_file)

In [75]:
from tqdm import tqdm
for i in tqdm(range(len(events_directions))):
    filterd_data(events_directions[i])
    

In [9]:
df_df = pd.DataFrame(event_times, columns=['filename','time_abs(%Y-%m-%dT%H:%M:%S.%f)','Trigger On (s)', 'Trigger Off (s)','image'])
df_df.head()

,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),Trigger On (s),Trigger Off (s),image
0,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv,2019-05-23T02:38:04.432000,2284.40,2332.15,images/XB.ELYSE.02.BHV.2019-05-23HR02_evid0041...
1,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv,2019-05-23T02:45:30.632000,2730.60,2770.50,images/XB.ELYSE.02.BHV.2019-05-23HR02_evid0041...
2,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv,2019-07-26T12:12:30.410000,750.40,798.85,images/XB.ELYSE.02.BHV.2019-07-26HR12_evid0033...
3,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv,2019-07-26T12:22:10.410000,1330.40,1370.55,images/XB.ELYSE.02.BHV.2019-07-26HR12_evid0033...
4,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv,2019-07-26T12:26:14.260000,1574.25,1612.40,images/XB.ELYSE.02.BHV.2019-07-26HR12_evid0033...


In [10]:
df_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1677 entries, 0 to 1676
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   filename                        1677 non-null   object 
 1   time_abs(%Y-%m-%dT%H:%M:%S.%f)  1677 non-null   object 
 2   Trigger On (s)                  1677 non-null   float64
 3   Trigger Off (s)                 1677 non-null   float64
 4   image                           1677 non-null   object 
dtypes: float64(2), object(3)
memory usage: 65.6+ KB


In [11]:
df_df['datetime'] = pd.to_datetime(df_df['time_abs(%Y-%m-%dT%H:%M:%S.%f)'])
df_df.head()

,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),Trigger On (s),Trigger Off (s),image,datetime
0,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv,2019-05-23T02:38:04.432000,2284.40,2332.15,images/XB.ELYSE.02.BHV.2019-05-23HR02_evid0041...,2019-05-23 02:38:04.432
1,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv,2019-05-23T02:45:30.632000,2730.60,2770.50,images/XB.ELYSE.02.BHV.2019-05-23HR02_evid0041...,2019-05-23 02:45:30.632
2,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv,2019-07-26T12:12:30.410000,750.40,798.85,images/XB.ELYSE.02.BHV.2019-07-26HR12_evid0033...,2019-07-26 12:12:30.410
3,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv,2019-07-26T12:22:10.410000,1330.40,1370.55,images/XB.ELYSE.02.BHV.2019-07-26HR12_evid0033...,2019-07-26 12:22:10.410
4,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv,2019-07-26T12:26:14.260000,1574.25,1612.40,images/XB.ELYSE.02.BHV.2019-07-26HR12_evid0033...,2019-07-26 12:26:14.260


In [12]:
df_df['date'] = df_df['datetime'].dt.date

In [13]:
df_df['time'] = df_df['datetime'].dt.time


In [15]:
df_df.head()

,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),Trigger On (s),Trigger Off (s),image,datetime,date,time
0,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv,2019-05-23T02:38:04.432000,2284.40,2332.15,images/XB.ELYSE.02.BHV.2019-05-23HR02_evid0041...,2019-05-23 02:38:04.432,2019-05-23,02:38:04.432000
1,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv,2019-05-23T02:45:30.632000,2730.60,2770.50,images/XB.ELYSE.02.BHV.2019-05-23HR02_evid0041...,2019-05-23 02:45:30.632,2019-05-23,02:45:30.632000
2,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv,2019-07-26T12:12:30.410000,750.40,798.85,images/XB.ELYSE.02.BHV.2019-07-26HR12_evid0033...,2019-07-26 12:12:30.410,2019-07-26,12:12:30.410000
3,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv,2019-07-26T12:22:10.410000,1330.40,1370.55,images/XB.ELYSE.02.BHV.2019-07-26HR12_evid0033...,2019-07-26 12:22:10.410,2019-07-26,12:22:10.410000
4,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv,2019-07-26T12:26:14.260000,1574.25,1612.40,images/XB.ELYSE.02.BHV.2019-07-26HR12_evid0033...,2019-07-26 12:26:14.260,2019-07-26,12:26:14.260000


In [22]:
df_df['sensor'] = df_df['filename'].str.split('.').str[1]
df_df['network'] = df_df['filename'].str.split('.').str[0]


In [23]:
df_df.head()

,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),Trigger On (s),Trigger Off (s),image,datetime,date,time,sensor,network
0,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv,2019-05-23T02:38:04.432000,2284.40,2332.15,images/XB.ELYSE.02.BHV.2019-05-23HR02_evid0041...,2019-05-23 02:38:04.432,2019-05-23,02:38:04.432000,ELYSE,XB
1,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv,2019-05-23T02:45:30.632000,2730.60,2770.50,images/XB.ELYSE.02.BHV.2019-05-23HR02_evid0041...,2019-05-23 02:45:30.632,2019-05-23,02:45:30.632000,ELYSE,XB
2,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv,2019-07-26T12:12:30.410000,750.40,798.85,images/XB.ELYSE.02.BHV.2019-07-26HR12_evid0033...,2019-07-26 12:12:30.410,2019-07-26,12:12:30.410000,ELYSE,XB
3,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv,2019-07-26T12:22:10.410000,1330.40,1370.55,images/XB.ELYSE.02.BHV.2019-07-26HR12_evid0033...,2019-07-26 12:22:10.410,2019-07-26,12:22:10.410000,ELYSE,XB
4,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv,2019-07-26T12:26:14.260000,1574.25,1612.40,images/XB.ELYSE.02.BHV.2019-07-26HR12_evid0033...,2019-07-26 12:26:14.260,2019-07-26,12:26:14.260000,ELYSE,XB


In [24]:
df_df['sensor'].value_counts()

sensor
s12      1530
s15        76
s16        49
ELYSE      22
Name: count, dtype: int64

In [25]:
df_df['latitude'] = np.nan 

In [26]:
df_df['longitude'] = np.nan 

In [29]:
df_df['mission_name'] = np.nan

In [30]:
def add_coordinates_and_name(row):
    if row['sensor'] == 'ELYSE':
        row['latitude'] = 4.502  # إحداثيات InSight
        row['longitude'] = 135.623
        row['mission_name'] = 'InSight'
    elif row['sensor'] == 's12':
        row['latitude'] = 3.01239  # إحداثيات Apollo 12
        row['longitude'] = -23.42157  # لاحظ أن الطول السالب يعني غرباً
        row['mission_name'] = 'Apollo 12'
    elif row['sensor'] == 's15':
        row['latitude'] = 26.13222  # إحداثيات Apollo 15
        row['longitude'] = 3.63386
        row['mission_name'] = 'Apollo 15'
    elif row['sensor'] == 's16':
        row['latitude'] = 8.97301  # إحداثيات Apollo 16
        row['longitude'] = 15.50019
        row['mission_name'] = 'Apollo 16'
    else:
        row['latitude'] = None
        row['longitude'] = None
        row['mission_name'] = None
    return row

In [31]:
df_df = df_df.apply(add_coordinates_and_name, axis=1)


In [32]:
df_df.head()

,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),Trigger On (s),Trigger Off (s),image,datetime,date,time,sensor,network,latitude,longitude,mission_name
0,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv,2019-05-23T02:38:04.432000,2284.40,2332.15,images/XB.ELYSE.02.BHV.2019-05-23HR02_evid0041...,2019-05-23 02:38:04.432,2019-05-23,02:38:04.432000,ELYSE,XB,4.502,135.623,InSight
1,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv,2019-05-23T02:45:30.632000,2730.60,2770.50,images/XB.ELYSE.02.BHV.2019-05-23HR02_evid0041...,2019-05-23 02:45:30.632,2019-05-23,02:45:30.632000,ELYSE,XB,4.502,135.623,InSight
2,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv,2019-07-26T12:12:30.410000,750.40,798.85,images/XB.ELYSE.02.BHV.2019-07-26HR12_evid0033...,2019-07-26 12:12:30.410,2019-07-26,12:12:30.410000,ELYSE,XB,4.502,135.623,InSight
3,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv,2019-07-26T12:22:10.410000,1330.40,1370.55,images/XB.ELYSE.02.BHV.2019-07-26HR12_evid0033...,2019-07-26 12:22:10.410,2019-07-26,12:22:10.410000,ELYSE,XB,4.502,135.623,InSight
4,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv,2019-07-26T12:26:14.260000,1574.25,1612.40,images/XB.ELYSE.02.BHV.2019-07-26HR12_evid0033...,2019-07-26 12:26:14.260,2019-07-26,12:26:14.260000,ELYSE,XB,4.502,135.623,InSight


In [36]:
def image_id(img):
    return img.split('/')[1]

In [37]:
df_df['image id'] = np.nan 

In [47]:
df_df['url'] = np.nan 

In [42]:
df_df['image id'] = df_df['image'].apply(image_id)

In [43]:
df_df['image id']

0       XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv.png
1       XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv.png
2       XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv.png
3       XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv.png
4       XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv.png
                             ...                       
1672     xa.s16.00.mhz.1977-04-17HR00_evid00249.csv.png
1673     xa.s16.00.mhz.1977-06-02HR00_evid00255.csv.png
1674     xa.s16.00.mhz.1977-06-02HR00_evid00255.csv.png
1675     xa.s16.00.mhz.1977-06-02HR00_evid00255.csv.png
1676     xa.s16.00.mhz.1977-06-02HR00_evid00255.csv.png
Name: image id, Length: 1677, dtype: object

In [45]:
urls = pd.read_csv('urls.csv')

In [46]:
urls.head()

,Image Name,Shareable Link
0,xa.s16.00.mhz.1977-06-02HR00_evid00255.csv.png,https://drive.google.com/file/d/1ORLQC0729NjPT...
1,xa.s16.00.mhz.1977-04-17HR00_evid00249.csv.png,https://drive.google.com/file/d/1Shrb9_SjyqsCk...
2,xa.s16.00.mhz.1975-03-26HR00_evid00186.csv.png,https://drive.google.com/file/d/1y1_DeHjEAfASi...
3,xa.s16.00.mhz.1975-02-19HR00_evid00180.csv.png,https://drive.google.com/file/d/1wHqzDuEDgp1Uq...
4,xa.s16.00.mhz.1974-12-25HR00_evid00174.csv.png,https://drive.google.com/file/d/1m2G7KQZVTR-xs...


In [48]:
for i in range(len(df_df)):
    im_id = df_df['image id'][i]
    for j in range(len(urls)):
        urls_id = urls['Image Name'][j]
        if im_id == urls_id:
            df_df['url'][i] = urls['Shareable Link'][j]
        else:
            continue 
    

C:\Users\El-Wattaneya\AppData\Local\Temp\ipykernel_18632\3311604866.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_df['url'][i] = urls['Shareable Link'][j]
C:\Users\El-Wattaneya\AppData\Local\Temp\ipykernel_18632\3311604866.py:6: Set

In [79]:
mag = []
def magnitude(file):
    csv_file = file
    data_cat = pd.read_csv(csv_file,encoding='utf-8')
    try: 
        csv_times = np.array(data_cat['time_rel(sec)'])
        csv_data = np.array(data_cat['velocity(m/s)'])
    except:
        csv_times = np.array(data_cat['rel_time(sec)'])
        csv_data = np.array(data_cat['velocity(c/s)'])
        
    filtered_data = bandpass_filter(csv_data)
    max_amplitude = np.max(np.abs(filtered_data))
    magnitude = np.log10(max_amplitude) + 3  #
    cft = classic_sta_lta(filtered_data, int(sta_len * sampling_rate), int(lta_len * sampling_rate))
    triggers = trigger_onset(cft, threshold_on, threshold_off)
    for i in range(len(triggers)):
        mag.append(magnitude)


In [80]:
for i in events_directions:
    magnitude(i)

In [81]:
len(mag)

1677

In [83]:
df_df['magnitude'] = mag

In [82]:
df_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1677 entries, 0 to 1676
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   filename                        1677 non-null   object        
 1   time_abs(%Y-%m-%dT%H:%M:%S.%f)  1677 non-null   object        
 2   Trigger On (s)                  1677 non-null   float64       
 3   Trigger Off (s)                 1677 non-null   float64       
 4   image                           1677 non-null   object        
 5   datetime                        1677 non-null   datetime64[ns]
 6   date                            1677 non-null   object        
 7   time                            1677 non-null   object        
 8   sensor                          1677 non-null   object        
 9   network                         1677 non-null   object        
 10  latitude                        1677 non-null   float64       
 11  long

In [94]:
df_df['time'][0].strftime('%H:%M:%S')

'02:38:04'

In [49]:
df_df.head()

,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),Trigger On (s),Trigger Off (s),image,datetime,date,time,sensor,network,latitude,longitude,mission_name,image id,url
0,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv,2019-05-23T02:38:04.432000,2284.40,2332.15,images/XB.ELYSE.02.BHV.2019-05-23HR02_evid0041...,2019-05-23 02:38:04.432,2019-05-23,02:38:04.432000,ELYSE,XB,4.502,135.623,InSight,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv.png,https://drive.google.com/file/d/1JXMSwhF2ptGkP...
1,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv,2019-05-23T02:45:30.632000,2730.60,2770.50,images/XB.ELYSE.02.BHV.2019-05-23HR02_evid0041...,2019-05-23 02:45:30.632,2019-05-23,02:45:30.632000,ELYSE,XB,4.502,135.623,InSight,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv.png,https://drive.google.com/file/d/1JXMSwhF2ptGkP...
2,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv,2019-07-26T12:12:30.410000,750.40,798.85,images/XB.ELYSE.02.BHV.2019-07-26HR12_evid0033...,2019-07-26 12:12:30.410,2019-07-26,12:12:30.410000,ELYSE,XB,4.502,135.623,InSight,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv.png,https://drive.google.com/file/d/1TQNi3PmY7O5mS...
3,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv,2019-07-26T12:22:10.410000,1330.40,1370.55,images/XB.ELYSE.02.BHV.2019-07-26HR12_evid0033...,2019-07-26 12:22:10.410,2019-07-26,12:22:10.410000,ELYSE,XB,4.502,135.623,InSight,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv.png,https://drive.google.com/file/d/1TQNi3PmY7O5mS...
4,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv,2019-07-26T12:26:14.260000,1574.25,1612.40,images/XB.ELYSE.02.BHV.2019-07-26HR12_evid0033...,2019-07-26 12:26:14.260,2019-07-26,12:26:14.260000,ELYSE,XB,4.502,135.623,InSight,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv.png,https://drive.google.com/file/d/1TQNi3PmY7O5mS...


In [95]:
import json

data_list = []

for i in range(len(df_df)):  
    new_dict  =  {'id':'.'.join(df_df['image id'][i].split('.')[0:-2]),
        'date': df_df['date'][i].strftime('%Y-%m-%d'),
        'time': df_df['time'][i].strftime('%H:%M:%S'),
        'Trigger On (s)': df_df['Trigger On (s)'][i],
        'Trigger Off (s)': df_df['Trigger Off (s)'][i],
        'time_stmp': df_df['time_abs(%Y-%m-%dT%H:%M:%S.%f)'][i],
        'sensor': df_df['sensor'][i],
        'network': df_df['network'][i],
        'latitude': df_df['latitude'][i],
        'longitude': df_df['longitude'][i],
        'mission_name': df_df['mission_name'][i],
         'magnitude': df_df['magnitude'][i] ,
        'url': df_df['url'][i]
                                }
    data_list.append(new_dict)



In [96]:
data_list

[{'id': 'XB.ELYSE.02.BHV.2019-05-23HR02_evid0041',
  'date': '2019-05-23',
  'time': '02:38:04',
  'Trigger On (s)': 2284.4,
  'Trigger Off (s)': 2332.15,
  'time_stmp': '2019-05-23T02:38:04.432000',
  'sensor': 'ELYSE',
  'network': 'XB',
  'latitude': 4.502,
  'longitude': 135.623,
  'mission_name': 'InSight',
  'magnitude': 6.342044173727068,
  'url': 'https://drive.google.com/file/d/1JXMSwhF2ptGkPCEZKj9HD32306zn1RPI/view?usp=drivesdk'},
 {'id': 'XB.ELYSE.02.BHV.2019-05-23HR02_evid0041',
  'date': '2019-05-23',
  'time': '02:45:30',
  'Trigger On (s)': 2730.6,
  'Trigger Off (s)': 2770.5,
  'time_stmp': '2019-05-23T02:45:30.632000',
  'sensor': 'ELYSE',
  'network': 'XB',
  'latitude': 4.502,
  'longitude': 135.623,
  'mission_name': 'InSight',
  'magnitude': 6.342044173727068,
  'url': 'https://drive.google.com/file/d/1JXMSwhF2ptGkPCEZKj9HD32306zn1RPI/view?usp=drivesdk'},
 {'id': 'XB.ELYSE.02.BHV.2019-07-26HR12_evid0033',
  'date': '2019-07-26',
  'time': '12:12:30',
  'Trigger On 

In [97]:
output_file = 'moon&mars_data.json'
with open(output_file, 'w') as json_file:
    json.dump(data_list, json_file, indent=4) 

In [98]:
output_file = 'moon&mars_data.csv'
df_df.to_csv(output_file, index=False)